In [1]:
# !unzip /content/News_Category_Dataset_v3.json.zip

Archive:  /content/News_Category_Dataset_v3.json.zip
  inflating: News_Category_Dataset_v3.json  


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

In [ ]:
# Load dataset (text, category)
df = pd.read_json("News_Category_Dataset_v3.json", lines=True)
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [ ]:
df.shape

(209527, 6)

In [ ]:
len(df.category.unique())

42

In [ ]:
df['category'].value_counts().max()

35602

In [ ]:
df['category'].value_counts().min()

1014

In [ ]:
df.isnull().sum()

,0
link,0
headline,0
category,0
short_description,0
authors,0
date,0


In [ ]:
df.category.unique()

array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT', 'EDUCATION', 'CRIME', 'SCIENCE', 'WELLNESS',
       'BUSINESS', 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA',
       'QUEER VOICES', 'HOME & LIVING', 'WOMEN', 'BLACK VOICES', 'TRAVEL',
       'MONEY', 'RELIGION', 'LATINO VOICES', 'IMPACT', 'WEDDINGS',
       'COLLEGE', 'PARENTS', 'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE',
       'HEALTHY LIVING', 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST',
       'FIFTY', 'ARTS', 'DIVORCE'], dtype=object)

In [ ]:
df['category'].max()

'WORLDPOST'

In [ ]:
df['category'].min()

'ARTS'

In [ ]:
df['category'].value_counts().max()

35602

In [ ]:
allowed = ['SCIENCE','SPORTS','TECH','BUSINESS','EDUCATION']
df = df[df['category'].isin(allowed)]

In [ ]:
df['category'].value_counts().max()

5992

In [ ]:
df['category'].value_counts().min()

1014

In [ ]:
df = (
    df.sort_values('category')  # optional, for clean order
      .groupby('category', group_keys=False)
      .head(df['category'].value_counts().min())
      .reset_index(drop=True)
)

In [ ]:
df['category'].value_counts()

,count
category,
BUSINESS,1014
EDUCATION,1014
SCIENCE,1014
SPORTS,1014
TECH,1014


In [ ]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffingtonpost.com/entry/outdoor-w...,Outdoor Workspace: The Next Workplace Frontier,BUSINESS,If you could take a vacation to anywhere in th...,"Amanda Schneider, ContributorI write about the...",2015-05-19
1,https://www.huffingtonpost.com/entry/open-carr...,Pro-Gun Group Backs Down After Chipotle Rally ...,BUSINESS,,Ben Hallman,2014-05-23
2,https://www.huffingtonpost.com/entry/volkswage...,Volkswagen's Emissions Scandal Just Got So Muc...,BUSINESS,The auto giant admitted to a cheating on a CO2...,"David Rising, AP",2015-11-04
3,https://www.huffingtonpost.com/entry/gm-recall...,Mother Finds Out Years Later That GM May Be Re...,BUSINESS,,Kira Brekke,2014-05-23
4,https://www.huffingtonpost.com/entry/abercromb...,Abercrombie To Get Slightly Less Obnoxious,BUSINESS,,,2014-05-22


In [ ]:
df = df.drop(columns=['link', 'authors', 'date'])

In [ ]:
df.head(2)

,headline,category,short_description
0,Outdoor Workspace: The Next Workplace Frontier,BUSINESS,If you could take a vacation to anywhere in th...
1,Pro-Gun Group Backs Down After Chipotle Rally ...,BUSINESS,


**Joining Data**

In [ ]:
df['text'] = df['headline'] + " " + df['short_description']

**Encode labels**

In [ ]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['category'])

**Splitting the dataset**

In [ ]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42
)

**Load TF Hub embedding model**

In [ ]:
embedding_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
embed = hub.load(embedding_url)  # Load raw TF function

**Convert text → vectors**

In [ ]:
def text_to_vector(texts):
    # Convert to list of strings and embed
    vectors = embed(tf.constant(texts))
    return vectors.numpy()

X_train_vec = text_to_vector(X_train_text.tolist())
X_test_vec = text_to_vector(X_test_text.tolist())

**Showing vector**

In [ ]:
text_to_vector(["Nahid Hasan"]).shape

(1, 512)

**Preprocess vectors**

In [ ]:
scaler = StandardScaler()
X_train_vec = scaler.fit_transform(X_train_vec)
X_test_vec = scaler.transform(X_test_vec)

**Shwing the classes**

In [ ]:
label_encoder.classes_

array(['BUSINESS', 'EDUCATION', 'SCIENCE', 'SPORTS', 'TECH'], dtype=object)

 Build ANN model

In [ ]:
model = tf.keras.Sequential([

    tf.keras.layers.Input(shape=(X_train_vec.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_25 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 66,309 (259.02 KB)

 Trainable params: 66,309 (259.02 KB)

 Non-trainable params: 0 (0.00 B)

**Train**

In [ ]:
history = model.fit(
    X_train_vec, y_train,
    validation_data=(X_test_vec, y_test),
    epochs=10,
    batch_size=32
)

Epoch 1/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9713 - loss: 0.1079 - val_accuracy: 0.8629 - val_loss: 0.5396
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9777 - loss: 0.0804 - val_accuracy: 0.8580 - val_loss: 0.5609
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9841 - loss: 0.0669 - val_accuracy: 0.8550 - val_loss: 0.5964
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9867 - loss: 0.0617 - val_accuracy: 0.8570 - val_loss: 0.6217
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9893 - loss: 0.0504 - val_accuracy: 0.8580 - val_loss: 0.6312
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9890 - loss: 0.0478 - val_accuracy: 0.8570 - val_loss: 0.6629
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9911 - loss: 0.0374 - val_accuracy: 0.8550 - val_loss: 0.6738
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9911 - loss: 0.0372 - val_accuracy: 0.

**Evaluate**

In [ ]:
loss, acc = model.evaluate(X_test_vec, y_test)
print(f"Test Accuracy: {acc:.2f}")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8333 - loss: 0.8184
Test Accuracy: 0.85


**Predict function**

In [ ]:
def predict_category(text):
    vec = text_to_vector([text])
    vec = scaler.transform(vec)
    pred = model.predict(vec)
    return label_encoder.inverse_transform([np.argmax(pred)])[0]

print(predict_category("Bitcoin price surges after new regulations"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
BUSINESS


**Testing the model**

In [ ]:
print(predict_category("Brewers win ninth straight with Collins’ walk-off homer; Verlander strikes out 3,500th batter; USC and Penn State defenders named to Bednarik Award watch list."))



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
SPORTS


**Save the model**

In [ ]:
# model.save("./news_model.h5")

**Save classes to JSON**

In [ ]:
# import json

# # Save classes to JSON
# with open('label_classes.json', 'w') as f:
#     json.dump(label_encoder.classes_.tolist(), f)